###### Project

In [44]:
import pandas as pd
#import sqlite3
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import math
import datetime as dt
import folium
import geopandas

spray_data = pd.read_csv(
    'Files/spray.csv',  # шлях до файлу, який містить дані
    sep=',',  # роздільник стовпців
    header=0  # номер рядка, що містить заголовок (якщо заголовка немає, можна передати None)
)

train_data = pd.read_csv(
    'Files/train.csv',  # шлях до файлу, який містить дані
    sep=',',  # роздільник стовпців
    
    header=0  # номер рядка, що містить заголовок (якщо заголовка немає, можна передати None)
)

weather_data = pd.read_csv(
    'Files/weather.csv',  # шлях до файлу, який містить дані
    sep=',',  # роздільник стовпців
    header=0  # номер рядка, що містить заголовок (якщо заголовка немає, можна передати None)
)

test_truncated_data = pd.read_excel(
    'Files/test_truncated.xlsx',
    sheet_name='test_truncated',  # прочитати всі листи
    usecols="A:J"
    #nrows=5
)
DataSets={1:spray_data,
         2:train_data,
         3:weather_data,
         4:test_truncated_data}

#test_truncated_data.head()
#dates=spray_data [['Date'],['Time']]
#train_data.groupby(train_data["Address"])
#dir(spray_data)
#spray_data.amin(axes='Latitude')
#test_truncated_data.info()
#train_data
#weather_data

In [106]:
weather_data

,Station,Date,Tmax,Tmin,Tavg,Depart,DewPoint,WetBulb,Heat,Cool,...,SnowFall,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed,day,month,year
0,1,2007-05-01,83,50,67,14,51,56,0,2,...,0.0,0.00,29.10,29.82,1.7,27,9.2,1,5,2007
1,2,2007-05-01,84,52,68,M,51,57,0,3,...,M,0.00,29.18,29.82,2.7,25,9.6,1,5,2007
2,1,2007-05-02,59,42,51,-3,42,47,14,0,...,0.0,0.00,29.38,30.09,13.0,4,13.4,2,5,2007
3,2,2007-05-02,60,43,52,M,42,47,13,0,...,M,0.00,29.44,30.08,13.3,2,13.4,2,5,2007
4,1,2007-05-03,66,46,56,2,40,48,9,0,...,0.0,0.00,29.39,30.12,11.7,7,11.9,3,5,2007
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2939,2,2014-10-29,49,40,45,M,34,42,20,0,...,M,0.00,29.42,30.07,8.5,29,9.0,29,10,2014
2940,1,2014-10-30,51,32,42,-4,34,40,23,0,...,0.0,0.00,29.34,30.09,5.1,24,5.5,30,10,2014
2941,2,2014-10-30,53,37,45,M,35,42,20,0,...,M,T,29.41,30.10,5.9,23,6.5,30,10,2014
2942,1,2014-10-31,47,33,40,-6,25,33,25,0,...,0.1,0.03,29.49,30.20,22.6,34,22.9,31,10,2014


In [47]:
geometry = geopandas.points_from_xy(weather_data.Longitude, train_data.Latitude)
geo_df = geopandas.GeoDataFrame(
    train_data, geometry=geometry
)

#geo_df.head()

# Create a geometry list from the GeoDataFrame
geo_df_list = [[point.xy[1][0], point.xy[0][0]] for point in geo_df.geometry]
#geo_df_list

In [ ]:
heat_data = [[point.xy[1][0], point.xy[0][0]] for point in geo_df.geometry]

heat_data
plugins.HeatMap(heat_data).add_to(map)

In [100]:
def numMosquitos(draft_year, draft_month):
    numMosquitos=train_data[(geo_df['Trap'] == geo_df.Trap[i]) & (geo_df['month']==draft_month) 
                   & (geo_df['year']==draft_year)]['WnvPresent'].sum()
    return str(numMosquitos)

In [105]:
draft_month=7
draft_year=2007


map_ = folium.Map(location=[train_data_adres['Latitude'].mean(),  train_data_adres['Longitude'].mean()],
               tiles="Stamen Terrain", zoom_start=10, control_scale=False)

# Iterate through list and add a marker for each volcano, color-coded by its type.
i = 0
for coordinates in geo_df_list:
    if (geo_df.month[i] == draft_month) and  (geo_df.year[i] == draft_year):
        wnvpresents=int(train_data[(geo_df['Trap'] == geo_df.Trap[i]) & (geo_df['month']==draft_month) 
                   & (geo_df['year']==draft_year)]['WnvPresent'].sum())
        if wnvpresents>0:
            if geo_df.PIPIENS[i] == True:
                type_color = "green"
            if (geo_df.RESTUANS[i] == True):
                type_color = "red"
                # Place the markers with the popup labels and data
            map_.add_child(
                folium.Marker(
                    location=coordinates,
                    popup=geo_df.Trap[i]+ "<br>"
                    +'wnvpresents '+ str(wnvpresents)+ "<br>"+'numMosquitos '+ str(numMosquitos(draft_year, draft_month)),
                    icon=folium.Icon(color="%s" % type_color)
                    )
            )
    i = i + 1
map_.save(f"{draft_month}_{draft_year}_map.html")